In [386]:
!pip install sktime

In [387]:
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
from time import sleep
from sklearn.utils import shuffle
from sklearn.metrics import accuracy_score
from sktime.classification.kernel_based import RocketClassifier
from sktime.datatypes import convert_to
from sktime.datatypes._panel._convert import from_nested_to_3d_numpy


In [388]:
nr_of_samples_in_one_batch = 100

In [389]:
circles = pd.read_csv("circles.txt", na_filter=False, sep=r"\s+", header=None, dtype=np.float64)

In [390]:
circles.iloc[0]

0     -17.58
1       0.00
2    1036.13
3      -0.95
4      -3.78
5       1.01
Name: 0, dtype: float64

In [391]:
def read_X_samples(X, nr, samples):
    return samples.iloc[(nr*X):(nr*X+X), :]
        

In [392]:
first = read_X_samples(nr_of_samples_in_one_batch, 0, circles)

In [393]:
first.shape

(100, 6)

In [394]:
print(first)

        0      1        2     3     4     5
0  -17.58   0.00  1036.13 -0.95 -3.78  1.01
1  -18.55   3.42  1044.92 -0.43 -3.87 -0.30
2   -6.84   0.49  1039.55 -1.10 -2.41 -0.12
3  -25.39   4.39  1038.57 -0.58  1.07 -0.58
4  -18.07   0.00  1054.20 -0.12 -0.79 -0.85
..    ...    ...      ...   ...   ...   ...
95 -16.60   0.49  1035.16 -1.10 -1.74 -0.46
96 -18.55   4.39  1027.83  0.03 -2.20 -0.34
97 -28.32  10.25  1048.83  0.34 -1.98 -0.95
98 -16.60   5.37  1046.88 -1.92 -1.80  1.55
99 -23.44   2.44  1042.48 -0.06 -0.40 -0.79

[100 rows x 6 columns]


In [395]:
s = circles.iloc[0]

In [396]:
dF = pd.DataFrame()

In [432]:
def addEntryToDataFrame(dF, data, label):
    new_entry = {f"feature_{i}": [data.iloc[:, i].values] for i in range(first.shape[1])}
    new_entry["label"] = [label]
    dF = pd.concat([dF, pd.DataFrame(new_entry)], ignore_index=True)
    return(dF)

In [434]:
def renameFeatures(dF):
    preferred_names = ["acc_x", "acc_y", "acc_z", "gyr_x", "gyr_y", "gyr_z", "label"]
    dF.columns = preferred_names[:len(preferred_names)]
    
    return dF

In [436]:
def addWholeDatasetToDataFrame(dF, data, label):
    nr_of_batches = round(data.dropna(how='all').shape[0] / nr_of_samples_in_one_batch)
    
    for i in range(nr_of_batches):
        batch = read_X_samples(nr_of_samples_in_one_batch, i, data)
        dF = addEntryToDataFrame(dF, batch, label)
    dF = renameFeatures(dF)
    return dF

In [438]:
def readFile_addToDataFrame(filename, dF, label_name):
    motion = pd.read_csv(filename, na_filter=False, sep=r"\s+", header=None, dtype=np.float64)
    dF = addWholeDatasetToDataFrame(dF, motion, label_name)
    return dF

In [440]:
dF_updown = pd.DataFrame()
dF_circles = pd.DataFrame()

In [442]:
dF_updown = readFile_addToDataFrame('up_down2.txt', dF_updown, 'Up and down')
dF_circles = readFile_addToDataFrame('circles2.txt', dF_circles, 'Circle')

In [444]:
dF_updown['acc_x'].dtype, dF_updown['acc_x'][0].dtype

(dtype('float64'), dtype('float64'))

In [446]:
dF_updown.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3300 entries, 0 to 3299
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   acc_x   3300 non-null   float64
 1   acc_y   3300 non-null   float64
 2   acc_z   3300 non-null   float64
 3   gyr_x   3300 non-null   float64
 4   gyr_y   3300 non-null   float64
 5   gyr_z   3300 non-null   float64
 6   label   3300 non-null   object 
dtypes: float64(6), object(1)
memory usage: 180.6+ KB


In [448]:
dF_circles

,acc_x,acc_y,acc_z,gyr_x,gyr_y,gyr_z,label
0,-42.97,1.95,1033.20,-1.37,-0.91,0.67,Circle
1,-50.78,2.44,1032.23,-1.34,-2.80,1.49,Circle
2,-50.29,0.00,1041.50,-0.43,-1.25,-0.98,Circle
3,-42.97,10.25,1035.64,-2.44,-1.25,-2.04,Circle
4,-36.13,7.81,1046.88,-1.52,-2.93,-1.71,Circle
...,...,...,...,...,...,...,...
3376,168.46,-847.17,1227.54,37.26,-1.34,-2.77,Circle
3377,750.49,-260.74,791.99,-4.73,-5.09,37.26,Circle
3378,-324.22,-368.16,1997.07,-9.70,-4.76,14.12,Circle
3379,140.62,-69.34,899.90,2.53,-1.89,22.04,Circle


In [405]:
def concatenate_dataFrames(dF1, dF2):
    frame = [dF1, dF2]
    return pd.concat(frame, ignore_index=True)

In [406]:
totalDataFrame = concatenate_dataFrames(dF_circles, dF_updown)

In [407]:
dF_circles.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34 entries, 0 to 33
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   acc_x   34 non-null     object
 1   acc_y   34 non-null     object
 2   acc_z   34 non-null     object
 3   gyr_x   34 non-null     object
 4   gyr_y   34 non-null     object
 5   gyr_z   34 non-null     object
 6   label   34 non-null     object
dtypes: object(7)
memory usage: 2.0+ KB


In [408]:
dF_updown.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33 entries, 0 to 32
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   acc_x   33 non-null     object
 1   acc_y   33 non-null     object
 2   acc_z   33 non-null     object
 3   gyr_x   33 non-null     object
 4   gyr_y   33 non-null     object
 5   gyr_z   33 non-null     object
 6   label   33 non-null     object
dtypes: object(7)
memory usage: 1.9+ KB


In [409]:
totalDataFrame.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 67 entries, 0 to 66
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   acc_x   67 non-null     object
 1   acc_y   67 non-null     object
 2   acc_z   67 non-null     object
 3   gyr_x   67 non-null     object
 4   gyr_y   67 non-null     object
 5   gyr_z   67 non-null     object
 6   label   67 non-null     object
dtypes: object(7)
memory usage: 3.8+ KB


In [410]:
display(totalDataFrame.iloc[:85])

,acc_x,acc_y,acc_z,gyr_x,gyr_y,gyr_z,label
0,"[-42.97, -50.78, -50.29, -42.97, -36.13, -40.5...","[1.95, 2.44, 0.0, 10.25, 7.81, 0.49, 0.98, 6.8...","[1033.2, 1032.23, 1041.5, 1035.64, 1046.88, 10...","[-1.37, -1.34, -0.43, -2.44, -1.52, -2.35, -3....","[-0.91, -2.8, -1.25, -1.25, -2.93, -0.24, -2.1...","[0.67, 1.49, -0.98, -2.04, -1.71, 0.09, -1.16,...",Circle
1,"[-57.62, -45.41, -48.34, -56.15, -62.5, -39.06...","[-2.44, 10.74, 5.86, 0.98, -5.37, 0.49, 6.84, ...","[1041.5, 1060.55, 1041.02, 1038.57, 1042.48, 1...","[0.3, -1.34, 0.15, -0.21, -1.37, -3.57, -3.41,...","[-1.55, -2.74, -2.32, -2.87, -0.7, -1.22, -2.3...","[-0.52, 1.16, -0.46, 0.46, 1.49, -2.9, -0.15, ...",Circle
2,"[-41.99, -48.83, -35.16, -48.34, -51.27, -48.3...","[-1.46, -7.32, -6.35, -2.93, -0.49, 5.86, 2.44...","[1042.97, 1042.48, 1041.5, 1045.41, 1050.78, 1...","[0.7, -2.01, 0.52, -0.06, 1.22, -1.95, -1.19, ...","[-1.1, -1.31, -1.46, -2.47, -0.82, 0.49, -0.98...","[1.62, -0.67, 1.01, -1.16, 1.65, -0.24, -0.61,...",Circle
3,"[-54.2, -45.9, -50.29, -40.04, -34.67, -44.43,...","[3.91, -8.3, -0.49, -0.98, 8.79, 5.86, -10.74,...","[1041.99, 1045.41, 1032.23, 1035.16, 1042.48, ...","[-0.95, -0.37, -1.13, -1.8, -0.4, -0.27, -0.37...","[0.09, -1.49, -1.25, 0.52, -0.21, -3.6, 0.49, ...","[-2.44, 1.07, -2.01, -0.67, 0.27, 1.65, -1.71,...",Circle
4,"[-42.48, -54.69, -56.64, -33.2, -45.9, -41.99,...","[-0.49, -7.81, -4.39, -7.81, 3.42, 12.7, -2.93...","[1039.06, 1046.39, 1042.48, 1045.9, 1035.64, 1...","[0.43, -1.71, -1.83, -2.71, -0.46, -1.55, -2.9...","[-1.34, 0.76, -1.98, 0.03, 0.82, -0.95, -0.37,...","[1.4, 0.61, -0.18, -1.22, -0.7, -0.12, -1.46, ...",Circle
...,...,...,...,...,...,...,...
62,"[-69.82, -174.8, -341.31, -406.74, -286.13, -2...","[71.78, -131.35, -372.56, -536.62, -589.84, -5...","[1024.9, 1546.39, 1828.13, 2005.86, 2131.35, 2...","[108.9, 108.11, 88.6, 68.35, 33.05, -2.01, -2....","[-67.93, -87.96, -101.89, -87.07, -53.81, -47....","[-2.93, 7.29, -13.63, -27.87, -21.16, -23.81, ...",Up and down
63,"[-122.56, -74.71, -38.57, 7.81, -0.98, -114.26...","[274.41, 166.5, 139.16, 107.42, 178.22, 345.21...","[259.77, 142.09, 30.76, 6.35, -49.8, -68.36, -...","[-65.61, -42.8, -30.52, -29.12, -16.04, 12.59,...","[30.58, 44.18, 50.64, 37.93, 4.79, -23.38, -37...","[23.02, 30.43, 36.89, 38.63, 21.52, -6.86, -17...",Up and down
64,"[-153.32, -193.36, -132.32, 10.25, -106.93, -2...","[-476.07, -490.23, -624.51, -532.71, -372.07, ...","[1956.54, 2165.53, 2293.95, 1994.63, 1751.95, ...","[10.82, -7.1, -30.21, -60.12, -97.71, -107.59,...","[6.37, 58.84, 31.65, -6.71, -15.37, -10.15, 8....","[17.68, 25.21, 24.18, 6.04, -8.11, -7.07, 8.45...",Up and down
65,"[-37.11, -13.18, -82.52, -35.16, -89.36, -174....","[330.57, 314.45, 230.47, 116.7, 53.22, -56.64,...","[8.3, 141.11, 514.16, 786.13, 1224.61, 1683.11...","[67.2, 100.18, 123.78, 126.16, 125.98, 125.61,...","[-1.13, -12.07, -28.81, -52.84, -56.01, -66.65...","[9.05, 13.38, 12.65, 0.98, -26.98, -38.72, -37...",Up and down


In [411]:
#for row in range(len(totalDataFrame)):
 #   print(totalDataFrame.iloc[row]) 

In [412]:
totalDataFrame_shuffled = shuffle(totalDataFrame)

In [413]:
totalDataFrame_shuffled


,acc_x,acc_y,acc_z,gyr_x,gyr_y,gyr_z,label
48,"[7.32, -106.93, -184.57, -131.84, -92.29, -141...","[38.09, 169.92, 345.21, 307.62, 125.98, 108.4,...","[161.13, 224.12, 156.74, 184.08, 309.57, 504.8...","[-26.04, -11.74, 23.63, 53.14, 66.62, 77.23, 9...","[-0.3, -20.76, -29.88, -22.74, -24.02, -27.13,...","[11.34, -6.34, -4.33, 6.68, 2.1, -24.27, -31.0...",Up and down
31,"[490.23, 420.9, 466.31, 315.92, 477.05, 538.57...","[265.14, 34.67, 218.26, 127.93, -208.5, -326.1...","[1049.8, 1147.95, 1049.8, 947.75, 1036.13, 843...","[-2.71, 5.52, -5.64, -1.16, 0.4, 0.61, -2.47, ...","[-3.99, 5.52, -0.52, -0.52, 4.36, 8.29, -1.31,...","[-59.6, -45.12, -23.32, -16.19, 3.6, 6.31, 33....",Circle
62,"[-69.82, -174.8, -341.31, -406.74, -286.13, -2...","[71.78, -131.35, -372.56, -536.62, -589.84, -5...","[1024.9, 1546.39, 1828.13, 2005.86, 2131.35, 2...","[108.9, 108.11, 88.6, 68.35, 33.05, -2.01, -2....","[-67.93, -87.96, -101.89, -87.07, -53.81, -47....","[-2.93, 7.29, -13.63, -27.87, -21.16, -23.81, ...",Up and down
49,"[-131.84, -149.41, -63.48, 1.46, -50.29, -114....","[-396.97, -531.74, -548.34, -413.57, -250.49, ...","[2068.85, 2260.74, 2242.68, 1916.5, 1638.67, 1...","[6.83, -24.02, -38.81, -67.74, -87.87, -96.37,...","[30.46, 72.16, 40.73, 11.01, 6.34, 9.94, 20.18...","[24.09, 31.8, 10.7, -5.4, -3.35, 2.26, 9.09, 8...",Up and down
64,"[-153.32, -193.36, -132.32, 10.25, -106.93, -2...","[-476.07, -490.23, -624.51, -532.71, -372.07, ...","[1956.54, 2165.53, 2293.95, 1994.63, 1751.95, ...","[10.82, -7.1, -30.21, -60.12, -97.71, -107.59,...","[6.37, 58.84, 31.65, -6.71, -15.37, -10.15, 8....","[17.68, 25.21, 24.18, 6.04, -8.11, -7.07, 8.45...",Up and down
...,...,...,...,...,...,...,...
11,"[355.96, 371.09, 487.79, 295.41, 164.55, 183.5...","[150.39, 58.59, 272.95, 181.64, -117.19, -331....","[963.87, 1219.24, 732.91, 1187.01, 1084.47, 10...","[-0.24, -2.44, 3.2, 0.61, -2.65, 10.3, 6.43, -...","[1.07, 4.66, -0.82, -6.37, 1.28, -1.16, 1.31, ...","[-28.69, -13.14, -10.18, 1.19, 13.32, 5.24, -1...",Circle
55,"[-70.8, -82.03, -121.58, -183.59, -190.43, -24...","[39.55, -147.95, -239.26, -478.52, -546.87, -5...","[895.02, 1350.59, 1789.06, 2081.05, 2229.49, 2...","[120.7, 108.96, 96.89, 77.59, 39.94, 13.32, 1....","[-24.48, -32.13, -50.58, -69.97, -69.73, -75.5...","[-0.4, -8.69, -12.87, -25.49, -33.9, -40.58, -...",Up and down
41,"[-55.18, -0.49, -69.34, -132.32, -20.51, -21.9...","[-76.66, -66.41, 81.05, 12.21, -305.66, 2.93, ...","[1049.8, 1070.31, 1029.79, 1090.33, 1274.41, 9...","[-2.9, 0.73, -4.18, -5.49, -1.71, -7.77, -4.73...","[-0.18, -0.3, 3.05, 0.98, -0.7, 0.09, 2.68, -4...","[-10.06, -19.45, -11.74, -5.09, -4.48, -11.07,...",Up and down
2,"[-41.99, -48.83, -35.16, -48.34, -51.27, -48.3...","[-1.46, -7.32, -6.35, -2.93, -0.49, 5.86, 2.44...","[1042.97, 1042.48, 1041.5, 1045.41, 1050.78, 1...","[0.7, -2.01, 0.52, -0.06, 1.22, -1.95, -1.19, ...","[-1.1, -1.31, -1.46, -2.47, -0.82, 0.49, -0.98...","[1.62, -0.67, 1.01, -1.16, 1.65, -0.24, -0.61,...",Circle


In [414]:
y_data = totalDataFrame_shuffled['label']

In [415]:
X_data = totalDataFrame_shuffled.drop('label', axis=1)

In [416]:
X_data

,acc_x,acc_y,acc_z,gyr_x,gyr_y,gyr_z
48,"[7.32, -106.93, -184.57, -131.84, -92.29, -141...","[38.09, 169.92, 345.21, 307.62, 125.98, 108.4,...","[161.13, 224.12, 156.74, 184.08, 309.57, 504.8...","[-26.04, -11.74, 23.63, 53.14, 66.62, 77.23, 9...","[-0.3, -20.76, -29.88, -22.74, -24.02, -27.13,...","[11.34, -6.34, -4.33, 6.68, 2.1, -24.27, -31.0..."
31,"[490.23, 420.9, 466.31, 315.92, 477.05, 538.57...","[265.14, 34.67, 218.26, 127.93, -208.5, -326.1...","[1049.8, 1147.95, 1049.8, 947.75, 1036.13, 843...","[-2.71, 5.52, -5.64, -1.16, 0.4, 0.61, -2.47, ...","[-3.99, 5.52, -0.52, -0.52, 4.36, 8.29, -1.31,...","[-59.6, -45.12, -23.32, -16.19, 3.6, 6.31, 33...."
62,"[-69.82, -174.8, -341.31, -406.74, -286.13, -2...","[71.78, -131.35, -372.56, -536.62, -589.84, -5...","[1024.9, 1546.39, 1828.13, 2005.86, 2131.35, 2...","[108.9, 108.11, 88.6, 68.35, 33.05, -2.01, -2....","[-67.93, -87.96, -101.89, -87.07, -53.81, -47....","[-2.93, 7.29, -13.63, -27.87, -21.16, -23.81, ..."
49,"[-131.84, -149.41, -63.48, 1.46, -50.29, -114....","[-396.97, -531.74, -548.34, -413.57, -250.49, ...","[2068.85, 2260.74, 2242.68, 1916.5, 1638.67, 1...","[6.83, -24.02, -38.81, -67.74, -87.87, -96.37,...","[30.46, 72.16, 40.73, 11.01, 6.34, 9.94, 20.18...","[24.09, 31.8, 10.7, -5.4, -3.35, 2.26, 9.09, 8..."
64,"[-153.32, -193.36, -132.32, 10.25, -106.93, -2...","[-476.07, -490.23, -624.51, -532.71, -372.07, ...","[1956.54, 2165.53, 2293.95, 1994.63, 1751.95, ...","[10.82, -7.1, -30.21, -60.12, -97.71, -107.59,...","[6.37, 58.84, 31.65, -6.71, -15.37, -10.15, 8....","[17.68, 25.21, 24.18, 6.04, -8.11, -7.07, 8.45..."
...,...,...,...,...,...,...
11,"[355.96, 371.09, 487.79, 295.41, 164.55, 183.5...","[150.39, 58.59, 272.95, 181.64, -117.19, -331....","[963.87, 1219.24, 732.91, 1187.01, 1084.47, 10...","[-0.24, -2.44, 3.2, 0.61, -2.65, 10.3, 6.43, -...","[1.07, 4.66, -0.82, -6.37, 1.28, -1.16, 1.31, ...","[-28.69, -13.14, -10.18, 1.19, 13.32, 5.24, -1..."
55,"[-70.8, -82.03, -121.58, -183.59, -190.43, -24...","[39.55, -147.95, -239.26, -478.52, -546.87, -5...","[895.02, 1350.59, 1789.06, 2081.05, 2229.49, 2...","[120.7, 108.96, 96.89, 77.59, 39.94, 13.32, 1....","[-24.48, -32.13, -50.58, -69.97, -69.73, -75.5...","[-0.4, -8.69, -12.87, -25.49, -33.9, -40.58, -..."
41,"[-55.18, -0.49, -69.34, -132.32, -20.51, -21.9...","[-76.66, -66.41, 81.05, 12.21, -305.66, 2.93, ...","[1049.8, 1070.31, 1029.79, 1090.33, 1274.41, 9...","[-2.9, 0.73, -4.18, -5.49, -1.71, -7.77, -4.73...","[-0.18, -0.3, 3.05, 0.98, -0.7, 0.09, 2.68, -4...","[-10.06, -19.45, -11.74, -5.09, -4.48, -11.07,..."
2,"[-41.99, -48.83, -35.16, -48.34, -51.27, -48.3...","[-1.46, -7.32, -6.35, -2.93, -0.49, 5.86, 2.44...","[1042.97, 1042.48, 1041.5, 1045.41, 1050.78, 1...","[0.7, -2.01, 0.52, -0.06, 1.22, -1.95, -1.19, ...","[-1.1, -1.31, -1.46, -2.47, -0.82, 0.49, -0.98...","[1.62, -0.67, 1.01, -1.16, 1.65, -0.24, -0.61,..."


In [417]:
X_data_train, X_data_test, y_data_train, y_data_test = train_test_split(
    X_data, y_data, test_size=0.2, random_state=42
)

In [418]:
X_data_train.head()

,acc_x,acc_y,acc_z,gyr_x,gyr_y,gyr_z
11,"[355.96, 371.09, 487.79, 295.41, 164.55, 183.5...","[150.39, 58.59, 272.95, 181.64, -117.19, -331....","[963.87, 1219.24, 732.91, 1187.01, 1084.47, 10...","[-0.24, -2.44, 3.2, 0.61, -2.65, 10.3, 6.43, -...","[1.07, 4.66, -0.82, -6.37, 1.28, -1.16, 1.31, ...","[-28.69, -13.14, -10.18, 1.19, 13.32, 5.24, -1..."
50,"[-160.64, -143.07, -54.2, -37.11, -17.58, 29.3...","[86.43, 156.74, 157.23, 160.16, 93.26, 1.46, 7...","[432.13, 340.82, 220.21, 128.42, 117.19, 33.69...","[-77.16, -61.34, -32.99, -16.68, 3.17, 15.34, ...","[16.25, 25.09, 23.48, 29.91, 32.16, 35.43, 26....","[1.52, 10.03, 27.23, 32.87, 31.68, 25.34, 20.7..."
0,"[-42.97, -50.78, -50.29, -42.97, -36.13, -40.5...","[1.95, 2.44, 0.0, 10.25, 7.81, 0.49, 0.98, 6.8...","[1033.2, 1032.23, 1041.5, 1035.64, 1046.88, 10...","[-1.37, -1.34, -0.43, -2.44, -1.52, -2.35, -3....","[-0.91, -2.8, -1.25, -1.25, -2.93, -0.24, -2.1...","[0.67, 1.49, -0.98, -2.04, -1.71, 0.09, -1.16,..."
8,"[-8.3, -18.55, -24.41, -15.63, -20.02, -23.44,...","[-4.39, -0.49, -5.37, -9.28, -7.81, 9.28, -0.4...","[1039.55, 1046.88, 1038.09, 1053.71, 1036.62, ...","[-1.89, 0.09, -1.4, -2.1, 1.19, -1.68, -1.01, ...","[-1.07, -1.37, -5.12, -1.55, -0.18, -0.76, -2....","[0.88, 0.3, 0.49, 0.88, -1.74, 2.13, 1.07, -3...."
57,"[-265.62, -341.8, -363.28, -370.12, -252.44, -...","[-313.48, -352.54, -479.98, -523.44, -376.95, ...","[1922.36, 1941.41, 2150.39, 2148.93, 1820.8, 1...","[29.42, 7.16, -9.6, -30.24, -60.27, -81.89, -9...","[-45.98, 35.7, 64.02, 34.79, 15.64, 24.18, 28....","[-17.35, 27.9, 30.27, 10.7, 1.01, -0.18, -8.2,..."


In [419]:
X_data_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 53 entries, 11 to 20
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   acc_x   53 non-null     object
 1   acc_y   53 non-null     object
 2   acc_z   53 non-null     object
 3   gyr_x   53 non-null     object
 4   gyr_y   53 non-null     object
 5   gyr_z   53 non-null     object
dtypes: object(6)
memory usage: 2.9+ KB


In [420]:
rocket = RocketClassifier(n_features_per_kernel=6, n_jobs=-1, random_state=42)

In [421]:
sktime.datatypes.check_raise(data, mtype)

NameError: name 'sktime' is not defined

In [ ]:
rocket.fit(X_data_train, y_data_train)